# Runtime MNIST CNN Autoencoder Example
Uses Conv2D and Deconv2D operations to create a simple auto-encoder. In this example, it is possible that it learns the trivial function. It's intention is more to see how the *tt.network.conv2d_transpose()* function works.

An image scale of [0, 1] is used here.

In [ ]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [ ]:
BATCH_SIZE = 32
REG_LAMBDA = 1e-8
INITIAL_LR = 0.01
LR_DECAY_STEP_INTERVAL = 10000
LR_DECAY_FACTOR = 0.5

In [ ]:
DATA_ROOT = "/work/sauterme/data"
dataset_train = tt.datasets.mnist.MNISTTrainDataset(DATA_ROOT)
dataset_valid = tt.datasets.mnist.MNISTValidDataset(DATA_ROOT)

### Model

In [ ]:
INPUT_BN = False
ENC_BN = False
DEC_BN = False

class SimpleCNNAutoencoderModel(tt.model.AbstractModel):    
    def __init__(self, reg_lambda=0.0):
        super(SimpleCNNAutoencoderModel, self).__init__(reg_lambda)
        
    @tt.utils.attr.override
    def inference(self, inputs, targets, feeds,
                  is_training, device_scope, memory_device):
        
        if INPUT_BN:
            inputs = tf.contrib.layers.batch_norm(inputs)
        
        with tf.variable_scope("Encoder"):
            # 1: Conv
            conv1 = tt.network.conv2d("Conv1", inputs,
                                      8, (5, 5), (2, 2),
                                      weight_init=tf.contrib.layers.xavier_initializer_conv2d(),
                                      bias_init=-0.1,
                                      regularizer=tf.contrib.layers.l2_regularizer(self.reg_lambda),
                                      activation=tf.nn.relu)
            
            if ENC_BN:
                conv1 = tf.contrib.layers.batch_norm(conv1)

            # 2: Conv
            conv2 = tt.network.conv2d("Conv2", conv1,
                                      16, (3, 3), (2, 2),
                                      weight_init=tf.contrib.layers.xavier_initializer_conv2d(),
                                      bias_init=-0.1,
                                      regularizer=tf.contrib.layers.l2_regularizer(self.reg_lambda),
                                      activation=tf.nn.relu)
            
            if ENC_BN:
                conv2 = tf.contrib.layers.batch_norm(conv2)
            
            encoder_out = conv2

        with tf.variable_scope("Decoder"):
            # 3: Deconv
            conv3t = tt.network.conv2d_transpose("Deconv1", encoder_out,
                                                 8, (3, 3), (2, 2),
                                                 weight_init=tt.init.bilinear_initializer(),
                                                 bias_init=-0.1,
                                                 regularizer=tf.contrib.layers.l2_regularizer(self.reg_lambda),
                                                 activation=tf.nn.relu)
            
            if DEC_BN:
                conv3t = tf.contrib.layers.batch_norm(conv3t)

            # 4: Deconv
            conv4t = tt.network.conv2d_transpose("Deconv2", conv3t,
                                                 1, (5, 5), (2, 2),
                                                 weight_init=tt.init.bilinear_initializer(), 
                                                 bias_init=-0.1,
                                                 regularizer=tf.contrib.layers.l2_regularizer(self.reg_lambda),
                                                 activation=tf.nn.sigmoid)
            decoder_out = conv4t
            
        return decoder_out
    
    @tt.utils.attr.override
    def loss(self, predictions, targets, device_scope):
        return tt.loss.bce(predictions, targets) + tt.loss.mgdl(predictions, targets)

### Training

In [ ]:
tt.hardware.set_cuda_devices([0])
runtime = tt.core.DefaultRuntime()
runtime.register_datasets(dataset_train, dataset_valid, None)
runtime.register_model(SimpleCNNAutoencoderModel(reg_lambda=REG_LAMBDA))
runtime.build(INITIAL_LR,
              LR_DECAY_STEP_INTERVAL,
              LR_DECAY_FACTOR,
              is_autoencoder=True,
              verbose=True)

In [ ]:
runtime.train(batch_size=BATCH_SIZE, steps=1000, display_step=50, do_checkpoints=False, do_summary=False)

### Evaluation

In [ ]:
x, _ = dataset_valid.get_batch(4)

tt.visualization.display_batch(x * 255, nrows=2, ncols=2, title="Input")

pred = runtime.predict(x)

tt.visualization.display_batch(pred * 255, nrows=2, ncols=2, title="Reconstruction")

In [ ]:
runtime.test(BATCH_SIZE)

### Terminate

In [ ]:
runtime.close()